<a href="https://colab.research.google.com/github/saralstalin/ResponseWithQueriedData/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [75]:
#install necessary packages
!pip install -q  openai mysql-connector gradio

In [76]:
#import needed modules
from google.colab import userdata
from openai import OpenAI
import gradio as gr
import mysql.connector

#create an instance of openai API client
OpenAIAPIKey = userdata.get('OpenAIAPIKey')
from openai import OpenAI
client = OpenAI(
    api_key=OpenAIAPIKey,
)

# call openAI API and get response message
def GetChatCompletion(sytemPrompt, userPrompt, model="gpt-4o-mini"):
  completions = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": userPrompt,
          }
          ,
          {
              "role": "system",
              "content": sytemPrompt,
          }
      ],
      model="gpt-4o-mini",
  )
  return completions.choices[0].message.content

# get database connection details from Secrets
myDBHost = userdata.get('host')
myDBUser = userdata.get('user')
myDBPassword = userdata.get('password')
myDBDatabase = userdata.get('database')

# create an instance of the database connection
myDatabaseConnection = mysql.connector.connect(
  host= myDBHost,
  user= myDBUser,
  password=myDBPassword,
  database=myDBDatabase
)

# execute database queries
def ExecuteQuery(query):
  result = []
  cursor = None
  try:
      cursor = myDatabaseConnection.cursor()
      cursor.execute(query)
      result = cursor.fetchall()
  except Exception as e:
      result = []
  finally:
      # Ensure the cursor is closed, even if an exception occurs
      if cursor:
          cursor.close()
      return result

# execute database queries
def ExecuteQuery(query):
  cursor = myDatabaseConnection.cursor()
  cursor.execute(query)
  result = cursor.fetchall()
  return result

# get list of all tables from database
def GetTables():
  return ExecuteQuery("SHOW TABLES")

# get relevant list of tables using LLM
def GetRelevantTables(listOfTables, userPrompt):
  systemPrompt = f"get relevant tables from the list of tables {listOfTables} that would contain data for the user prompt. Do not explain. Make it comma separated list."
  return GetChatCompletion(systemPrompt, userPrompt)

# get schema of relevant tables
def GetSchemaofRelevantTables(relevantTables):
  relevantTableList = relevantTables.split(',')
  schema = ""
  for table in relevantTableList:
    result = ExecuteQuery(f"SHOW CREATE TABLE {table}")
    if len(result) > 0:
      schema += result[0][1].replace('`', '') + "\n\n"
  return schema

# generate sql query using LLM
def GenerateSQLQuery(relevantTableSchema, userPrompt):
  systemPrompt = f"Generate an SQL Query using the schema of the tables {relevantTableSchema} for the user prompt. Do not explain. Do not invent new tables. Do not create code tags."
  return GetChatCompletion(systemPrompt, userPrompt)

# format response using LLM
def FormatResponse(queryResults, userPrompt):
  systemPrompt = f"Format a response using the data {queryResults} to the user prompt. Do not invent information, respond only when you are sure otherwise";
  return GetChatCompletion(systemPrompt, userPrompt)

# run steps to generate chat response
def Chatbot(userPrompt):
  listOfTables = GetTables()
  print(listOfTables)
  relevantTables = GetRelevantTables(listOfTables, userPrompt)
  relevantTableSchema = GetSchemaofRelevantTables(relevantTables)
  query = GenerateSQLQuery(relevantTableSchema, userPrompt)
  queryResults = ExecuteQuery(query)
  response = FormatResponse(queryResults, userPrompt)
  return response

#gradio interface
gradioInterface = gr.Interface(
    fn=Chatbot,
    inputs= gr.Textbox(label="Question"),
    outputs="text",
    title="Chatbot")

# Caution share =True will make the interface publicly available
gradioInterface.launch(share=True)



Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://e3f7debbeb8eb0522f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
